In [1]:
import requests
import csv
from datetime import datetime

keys, transactions = None, []
current_time = 1654016400
last_transactions = set()

while True:
    query = """
    {
        repays(where: {timestamp_lte: """ + str(current_time) + """}, first: 1000, orderBy: timestamp, orderDirection: desc) {
            id
            pool {
                id
            }
            user {
                id
            }
            repayer {
                id
            }
            reserve {
                symbol
            }
            amount
            timestamp
        }
    }

    """
    response = requests.post('https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
                                '',
                                json={'query': query})
    if response.status_code != 200:
        print("Problem reading from timestamp", current_time, ":", response.status_code)
        continue
    
    try:
        data = response.json()["data"]["repays"]
    except Exception:
        print("Error at timestamp", current_time)
        continue

    if len(data) == 0:
        break
    
    if keys is None:
        keys = data[0].keys()

    index = 0
    try:
        while data[index]["id"] in last_transactions:
            index += 1
    except IndexError:
            current_time -= 1
            continue
    print(len(data) - index, "transactions found at timestamp", current_time)

    for transaction in data[index:]:
        transaction["user"] = transaction["user"]["id"]
        transaction["pool"] = transaction["pool"]["id"]
        transaction["repayer"] = transaction["repayer"]["id"]
        transaction["reserve"] = transaction["reserve"]["symbol"]
        transactions.append(transaction)

    current_time = int(data[-1]["timestamp"])
    index = -1
    last_transactions = set()
    while data[index]["timestamp"] == data[-1]["timestamp"]:
        last_transactions.add(data[index]["id"])
        index -= 1

with open('repay.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(transactions)
        

1000 transactions found at timestamp 1654016400
999 transactions found at timestamp 1653554206
999 transactions found at timestamp 1652895125
999 transactions found at timestamp 1652496461
999 transactions found at timestamp 1652347481
999 transactions found at timestamp 1652302269
998 transactions found at timestamp 1652143034
999 transactions found at timestamp 1652046378
999 transactions found at timestamp 1651776420
999 transactions found at timestamp 1651120439
999 transactions found at timestamp 1650332372
999 transactions found at timestamp 1649616115
999 transactions found at timestamp 1648677429
999 transactions found at timestamp 1647788819
999 transactions found at timestamp 1646828449
999 transactions found at timestamp 1646008756
999 transactions found at timestamp 1645515151
999 transactions found at timestamp 1644672436
999 transactions found at timestamp 1643651586
998 transactions found at timestamp 1643022195
999 transactions found at timestamp 1642840330
999 transact

In [2]:
!wc -l repay.csv
# 103903

105267 repay.csv


In [3]:
from google.colab import files
files.download('repay.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>